In [ ]:
import struct
import numpy
from write.pointer import Pointer
from write.pusher import Pusher
from write.key import Key

In [ ]:
#Put all bytes from target file into memmap array
target_file = numpy.memmap(filename = "tests/samples/dummy2.root", mode = "r", dtype = numpy.uint8, shape = (928,))

In [ ]:
_format1       = ">4si"
_format2_small = ">iiiiiiBiii18s"
_format2_big   = ">iqqiiiBiqi18s"
_format3       = ">hIIii"
_format4_small = ">iii"
_format4_big   = ">qqq"
_format5       = ">i"
_format_small  = ">ihiIhhii"
_format_big    = ">ihiIhhqq"
_startcheck_format = ">IH"

In [ ]:
filename = "dummy2.root"
bytename = filename.encode("utf-8")

In [ ]:
#New file
file = numpy.memmap(filename = filename, dtype = numpy.uint8, mode = "w+", shape = (1,))
#In the case of a 1D array, the shape is the size of the file
#In ROOT files, the fEND corresponds to the size of the file
#fEND for ~/Downloads/uproot/tests/samples/dummy.root is 928

In [ ]:
#Working on just replacing bytes on target_file
file = numpy.memmap(filename = filename, dtype = numpy.uint8, mode = "w+", shape = (928,))
file[:] = target_file[:]

In [ ]:
pointer = Pointer(0)
pusher = Pusher(file)

In [ ]:
s1 = "Hello World"
s1 = s1.encode("utf-8")

s2 = "Pratyush"
s2 = s2.encode("utf-8")

In [ ]:
#Header bytes
fCompress = 0 #Constant for now
fSeekInfo = 416 #FIXME

from write.header import Header
header = Header(bytename, fCompress, fSeekInfo)
pusher.head_push(pointer, header)

In [ ]:
#Streamer Key
from write.first_key import First_Key
pointer = Pointer(header.fSeekInfo)
pointcheck = pointer.index
key = First_Key(pointer, 0)
pusher.keyer(pointer, key)

key.fKeylen = pointer.index - pointcheck
key.fNbytes = key.fKeylen + key.fObjlen
pusher.keyer(Pointer(pointcheck), key)

if pointer.index>header.fEND:
    header.fEND = pointer.index
    header.fSeekFree = pointer.index
    
header.fNbytesInfo = key.fNbytes
pusher.head_push(Pointer(0), header)

In [ ]:
from write.TObjString.streamers import TObjString
tobjstring = TObjString(pusher, pointer)
tobjstring.write()

if pointer.index>header.fEND:
    header.fEND = pointer.index
    header.fSeekFree = pointer.index
    
pusher.head_push(Pointer(0), header)

In [ ]:
fSeekKeys = pointer.index

In [ ]:
#Tkey instance
from write.begin_key import Begin_Key
pointer = Pointer(header.fBEGIN)
pointcheck = pointer.index
fName = bytename
key = Begin_Key(fName)
pusher.keyer(pointer, key)

key.fKeylen = pointer.index - pointcheck
key.fObjlen = key.fNbytes - key.fKeylen
pusher.keyer(Pointer(pointcheck), key)

if pointer.index>header.fEND:
    header.fEND = pointer.index
    header.fSeekFree = pointer.index
    
pusher.head_push(Pointer(0), header)

In [ ]:
#Junk 
pusher.stringer(pointer, fName)

if pointer.index>header.fEND:
    header.fEND = pointer.index
    header.fSeekFree = pointer.index
    
pusher.head_push(Pointer(0), header)

In [ ]:
#Directory Information
from write.directoryinfo import DirectoryInfo
directory_pointcheck = pointer.index
fNbytesKeys = 0
fNbytesName = header.fNbytesName
directory = DirectoryInfo(fNbytesKeys, fNbytesName, fSeekKeys)
pusher.director(pointer, directory)

if pointer.index>header.fEND:
    header.fEND = pointer.index
    header.fSeekFree = pointer.index
    
pusher.head_push(Pointer(0), header)

directory_end = pointer.index

In [ ]:
#Header Key
pointer = Pointer(fSeekKeys)
head_key_pointcheck = pointer.index
fVersion = 4
fNbytes = directory.fNbytesKeys
fObjlen = 0
fDatime = 1573188772
fKeylen = 0
fCycle = 1
fSeekKey = directory.fSeekKeys
fSeekPdir = 100
fClassName = b'TFile'
fName = bytename
fTitle = b''
packer = _format_small
head_key = Key(packer, fNbytes, fVersion, fObjlen, fDatime, fKeylen, fCycle, fSeekKey, fSeekPdir, fClassName, fName, fTitle)
pusher.keyer(pointer, head_key)

head_key_end = pointer.index

if pointer.index>header.fEND:
    header.fEND = pointer.index
    header.fSeekFree = pointer.index
    
pusher.head_push(Pointer(0), header)

In [ ]:
#Read number of keys
nkeys = 2
packer = _format5
pusher.numbers(pointer, packer, nkeys)

if pointer.index>header.fEND:
    header.fEND = pointer.index
    header.fSeekFree = pointer.index
    
pusher.head_push(Pointer(0), header)

In [ ]:
#Tkey instance 
pointcheck = pointer.index
fVersion = 4
fObjlen = 17 + len(s1) #Length of TObjString Branch Streamer
fDatime = 1573188772
fKeylen = 0
fNbytes = fObjlen + fKeylen
fCycle = 1
fSeekKey = directory_end + 30
fSeekPdir = 100
packer = _format_small
fClassName = b'TObjString'
fName = s1
fTitle = b'Collectable string class'

tkey1 = Key(packer, fNbytes, fVersion, fObjlen, fDatime, fKeylen, fCycle, fSeekKey, fSeekPdir, fClassName, fName, fTitle)
pusher.keyer(pointer, tkey1)

tkey1.fKeylen = pointer.index - pointcheck
tkey1.fNbytes = tkey1.fKeylen + tkey1.fObjlen
pusher.keyer(Pointer(pointcheck), tkey1)

if pointer.index>header.fEND:
    header.fEND = pointer.index
    header.fSeekFree = pointer.index
    
pusher.head_push(Pointer(0), header)

In [ ]:
#Junk
lowpointer = Pointer(fSeekKey)
pusher.keyer(lowpointer, tkey1)

In [ ]:
#TObjString Streamer - Have to get from branch
from write.TObjString.stringobject import StringObject
stringobject = StringObject(s1)
pusher.push_object(lowpointer, stringobject)

In [ ]:
#Tkey instance 
pointcheck = pointer.index
fVersion = 4
fObjlen = 17 + len(s2) #Length of TObjString Branch Streamer
fDatime = 1576183015
fKeylen = 0
fNbytes = fObjlen + fKeylen
fCycle = 1
fSeekKey = lowpointer.index
fSeekPdir = 100
packer = _format_small
fClassName = b'TObjString'
fName = s2
fTitle = b'Collectable string class'

tkey2 = Key(packer, fNbytes, fVersion, fObjlen, fDatime, fKeylen, fCycle, fSeekKey, fSeekPdir, fClassName, fName, fTitle)
pusher.keyer(pointer, tkey2)

tkey2.fKeylen = pointer.index - pointcheck
tkey2.fNbytes = tkey2.fKeylen + tkey2.fObjlen
pusher.keyer(Pointer(pointcheck), tkey2)

if pointer.index>header.fEND:
    header.fEND = pointer.index
    header.fSeekFree = pointer.index
    
pusher.head_push(Pointer(0), header)

In [ ]:
#Replacing Values
directory.fNbytesKeys = header.fEND - fSeekKeys
pusher.director(Pointer(directory_pointcheck), directory)

head_key.fNbytes = directory.fNbytesKeys
head_key.fKeylen = head_key_end - head_key_pointcheck
head_key.fObjlen = head_key.fNbytes - head_key.fKeylen
pusher.keyer(Pointer(head_key_pointcheck), head_key)

In [ ]:
#Junk
lowpointer = Pointer(fSeekKey)
pusher.keyer(lowpointer, tkey2)

In [ ]:
#TObjString Streamer - Have to get from branch
from write.TObjString.stringobject import StringObject
stringobject = StringObject(s2)
pusher.push_object(lowpointer, stringobject)

In [ ]:
#Examine bytes of target file
f = open("tests/samples/dummy2.root", "rb")
count = 0
try:
    byte = f.read(1)
    while byte != "":
        print (count, " ", byte)
        count = count + 1
        byte = f.read(1)
        if count == header.fEND:
            break
finally:
    f.close()

In [ ]:
#Examine bytes of our file
f = open(filename, "rb")
count = 0
try:
    byte = f.read(1)
    while byte != "":
        print (count, " ", byte)
        count = count + 1
        byte = f.read(1)
        if count == header.fEND:
            break
finally:
    f.close()

In [ ]:
#Examining target_file
target_file[:count]

In [ ]:
#Examining our file
file[:count]

In [ ]:
#Test - Debug target file with uproot
import uproot
uproot.open("tests/samples/dummy.root")

In [ ]:
#Test - Check hexdump of target file
!hexdump -C ~/Downloads/uproot/tests/samples/dummy.root

In [ ]:
#Test - uproot is reading is reading our file correctly
import uproot
uproot.open("uprootTObjString.root")

In [ ]:
#Test - Check size of file
import os
statinfo_target = os.stat("tests/samples/dummy.root")
print ("Size of target file is ", statinfo_target.st_size)
statinfo_copy = os.stat('uprootTObjString.root')
print ("Size of written file is ", statinfo_copy.st_size)